# Loading Movies Data

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os, time, json, math
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [114]:
#title_basic = ("https://datasets.imdbws.com/title.basics.tsv.gz")
#title_akas = ("https://datasets.imdbws.com/title.akas.tsv.gz")
#title_ratings = ("https://datasets.imdbws.com/title.ratings.tsv.gz")

In [4]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)

In [5]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy


In [5]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94229 entries, 0 to 94228
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          94229 non-null  object 
 1   titleType       94229 non-null  object 
 2   primaryTitle    94229 non-null  object 
 3   originalTitle   94229 non-null  object 
 4   isAdult         94229 non-null  int64  
 5   startYear       94229 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  94229 non-null  int64  
 8   genres          94229 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 6.5+ MB


In [6]:
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)

In [7]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [8]:
rating = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)

In [9]:
rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1905
1,tt0000002,5.9,256
2,tt0000005,6.2,2517
3,tt0000006,5.2,173
4,tt0000007,5.4,783


# Cleaning Data

In [121]:
basics = basics.replace({'\\N':np.nan})

In [75]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [123]:
akas = akas.replace({'\\N':np.nan})

In [125]:
rating = rating.replace({'\\N':np.nan})

In [127]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [128]:
basics.dropna(subset=['genres'], inplace=True)

In [134]:
basics = basics.loc[basics['titleType'] == 'movie']

In [151]:
basics = basics.loc[basics['genres'] != 'Documentary']

In [165]:
basics = basics.loc[basics['startYear'].between('2000','2022', inclusive=True)]

C:\Users\jonny\AppData\Local\Temp\ipykernel_18588\1918233108.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  basics = basics.loc[basics['startYear'].between('2000','2022', inclusive=True)]


In [169]:
akas = akas.loc[akas['region'] == 'US']

In [85]:
keepers = basics['tconst'].isin(akas['titleId'])
keepers

0          True
1          True
2          True
3         False
4          True
          ...  
163953     True
163954     True
163955    False
163956     True
163957    False
Name: tconst, Length: 163958, dtype: bool

In [86]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
7,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy
...,...,...,...,...,...,...,...,...,...
163950,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
163951,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
163953,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
163954,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [87]:
keepers2 = rating['tconst'].isin(akas['titleId'])
keepers2

0          True
1          True
4          True
5          True
6          True
           ... 
1262939    True
1262946    True
1262947    True
1262951    True
1262965    True
Name: tconst, Length: 475727, dtype: bool

In [88]:
rating = rating[keepers2]
rating

,tconst,averageRating,numVotes
0,tt0000001,5.7,1905
1,tt0000002,5.9,256
4,tt0000005,6.2,2517
5,tt0000006,5.2,173
6,tt0000007,5.4,783
...,...,...,...
1262939,tt9916204,8.1,242
1262946,tt9916348,8.5,17
1262947,tt9916362,6.4,4814
1262951,tt9916428,3.8,14


In [106]:
#checking if 'startYear' is numeric
basics[basics['startYear'] == 2020]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2020,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2020,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2020,NaN,100,"Comedy,Horror,Sci-Fi"
7,tt0094859,movie,Chief Zabu,Chief Zabu,0,2020,NaN,74,Comedy
...,...,...,...,...,...,...,...,...,...
163950,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2020,NaN,74,Drama
163951,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2020,NaN,97,"Comedy,Drama,Fantasy"
163953,tt9916170,movie,The Rehearsal,O Ensaio,0,2020,NaN,51,Drama
163954,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


# Exporting Data

In [94]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [93]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [92]:
rating.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [100]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94229 entries, 0 to 163956
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          94229 non-null  object 
 1   titleType       94229 non-null  object 
 2   primaryTitle    94229 non-null  object 
 3   originalTitle   94229 non-null  object 
 4   isAdult         94229 non-null  int64  
 5   startYear       94229 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  94229 non-null  int64  
 8   genres          94229 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 7.2+ MB


In [99]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1342190 entries, 0 to 1342189
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1342190 non-null  object 
 1   ordering         1342190 non-null  int64  
 2   title            1342190 non-null  object 
 3   region           1342190 non-null  object 
 4   language         3676 non-null     object 
 5   types            963271 non-null   object 
 6   attributes       44717 non-null    object 
 7   isOriginalTitle  1340815 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 81.9+ MB


In [98]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475727 entries, 0 to 1262965
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         475727 non-null  object 
 1   averageRating  475727 non-null  float64
 2   numVotes       475727 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.5+ MB


# Specifications - Financial Data - TMDB

In [20]:
with open('C:/Program Files/Git/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict # tmdb_api.json
login.keys()

dict_keys(['client-id', 'api-key'])

In [21]:
tmdb.API_KEY =  login['api-key']

In [22]:
#making function for search of movie with ID and includes movie certification
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

In [23]:
test = get_movie_with_rating("tt0848228") #testing id 1 movie
test

{'adult': False,
 'backdrop_path': '/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 245.974,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [14]:
pd.DataFrame(data=[test])

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.706,27397,PG-13


In [24]:
test2 = get_movie_with_rating("tt0332280") #testing id 2 movie
test2

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 70.325,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [25]:
pd.DataFrame(data=[test2])

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.88,9746,PG-13


# Breaking down by year 2000-2001

In [26]:
FOLDER = "Data"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [27]:
YEARS_TO_GET = [2000,2001]

In [28]:
YEARS_TO_GET = [2000,2001]
# Start of OUTER loop for saving each year 2000-2001 (Testing)
    #"""Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # save an empty dict with just "imdb_id" to the new json file.
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    # Loading in the dataframe from project part 1 as basics:
    basics = pd.read_csv('Data/title_basics.csv.gz')   
    #Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filtering out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
        
# INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
                # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
                    # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            print(e)
            
        final_year_df = pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",  
                             compression="gzip", index=False)
        
    final_year_df = pd.read_json(JSON_FILE)  
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", 
                             compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1503 [00:00<?, ?it/s]

name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0116628?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0118710?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0119966?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_j

name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt01

name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0208910?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' i

...

name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0290766?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0291237?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not def

name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0306500?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0307283?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0308556?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0308727?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
404 Client Er

404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0376057?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0386661?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0387555?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0391219?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0391402?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0391422?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0391512?api_key=9bebc24

404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt10707030?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt10924988?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt10933608?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt11700882?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt1198203?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt12024386?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt12921142?api_key=9

404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt6246810?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt6258040?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt6442762?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt6920966?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt7145146?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt7529862?api_key=9bebc24cb46b091165c091a8e0f1a17d
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt7795706?api_key=9bebc24cb46b091165c091a8e0f1a17d
name 'write_json' is not defined
name 'write_json' is not defined
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/t